In [1]:
# Place all your imports here
import yfinance as yf
import pandas_datareader.data as web
import numpy as np
import statsmodels.api as sm
import pandas as pd

This exercise involves estimating CAPM on: Gold, Exxon, General Electric, IBM, Microsoft, and Walmart. After downloading the price data (for these assets and also for the S&P 500 index and the 4-week treasury bill), you'll conduct the following steps:
- Calculate the monthly returns for each.
- Letting $r_{it}-r_{ft}$ represent the excess return on asset $i$ and $r_{mt}-r_{ft}$ represent the excess return on the market (proxied by the S&P 500), estimate: $r_{it}-r_{ft} = \alpha + \beta(r_{mt}-r_{ft}) + u_t$.
- For each asset test the restrictions $\alpha = 0$ and $\beta = 0$ both individually and jointly.

In [2]:
# This step takes care of downloading the prices
# Adjusted stock prices from Yahoo! Finance
data = yf.download('^GSPC XOM GE IBM MSFT WMT GC=F', start='2005-01-01', end='2020-01-31', interval='1mo')['Adj Close']
# 4-week Treasury Bill rate from FRED
tbill = web.DataReader(['TB4WK'], 'fred', start='2005-01-01', end='2019-12-31')

[*********************100%***********************]  7 of 7 completed


Create a new dataframe called *ret* that contains the continuous compounded monthly return for each asset in *data*. This should only take one line.

In [3]:
# Your code here
ret = np.log(data/data.shift(1))
ret

,GC=F,GE,IBM,MSFT,WMT,XOM,^GSPC
Date,,,,,,,
2005-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-01,0.034257,-0.026077,-0.009032,-0.043553,-0.015191,0.204522,0.018727
2005-03-01,-0.018031,0.030395,-0.011141,-0.037062,-0.029495,-0.055502,-0.019303
2005-04-01,0.014589,0.003874,-0.179306,0.045692,-0.058155,-0.044079,-0.020314
2005-05-01,NaN,0.007705,-0.010926,0.019570,0.001908,-0.014660,0.029512
...,...,...,...,...,...,...,...
2019-09-01,NaN,0.080322,0.082147,0.011775,0.042865,0.042987,0.017035
2019-10-01,NaN,0.111128,-0.083803,0.030739,-0.012037,-0.044008,0.020226
2019-11-01,-0.030772,0.121561,0.005370,0.054364,0.015486,0.008253,0.033480


If you take a look at *ret*, you'll notice that the first row is NaN and the first return is for Feb 2005. This is expected since the first return will be based on the prices in the first two rows. However, since the prices are for the beginning of each month, it means that the first return should really be for Jan 2005. Fix this. After you're done, what was previously the return for Feb 2005 should be now assigned to Jan 2005 and so on for all the remaining months.

In [4]:
# Your code here
ret_2 = ret.shift(-1)
ret_2

,GC=F,GE,IBM,MSFT,WMT,XOM,^GSPC
Date,,,,,,,
2005-01-01,0.034257,-0.026077,-0.009032,-0.043553,-0.015191,0.204522,0.018727
2005-02-01,-0.018031,0.030395,-0.011141,-0.037062,-0.029495,-0.055502,-0.019303
2005-03-01,0.014589,0.003874,-0.179306,0.045692,-0.058155,-0.044079,-0.020314
2005-04-01,NaN,0.007705,-0.010926,0.019570,0.001908,-0.014660,0.029512
2005-05-01,NaN,-0.051466,-0.015378,-0.034752,0.023526,0.027426,-0.000143
...,...,...,...,...,...,...,...
2019-09-01,NaN,0.111128,-0.083803,0.030739,-0.012037,-0.044008,0.020226
2019-10-01,-0.030772,0.121561,0.005370,0.054364,0.015486,0.008253,0.033480
2019-11-01,NaN,-0.009809,0.008688,0.044298,-0.002102,0.035918,0.028189


Next, let's get to the *tbill* data. Treasury bills are quoted on a 360-day discount basis. We have the 4-week Treasury Bill rate and the rate is quoted on an annualized basis. To convert into the 4-week rate (which is what we need since all our data are monthly), scale the quoted rate by multiplying by 4/52 (or 1/13). The other adjustment to make is that the quote is in percentage while our other returns data is in decimal format. So you'll need to fix this too. After fixing the Treasury rate, join that series with the *ret* dataframe. All of this can be done in one step.

In [5]:
# Your code here
tbill = tbill.mul(1/13).div(100)
tbill

,TB4WK
DATE,
2005-01-01,0.001531
2005-02-01,0.001785
2005-03-01,0.002000
2005-04-01,0.001992
2005-05-01,0.001992
...,...
2019-08-01,0.001562
2019-09-01,0.001500
2019-10-01,0.001308


Now you're ready to estimate CAPM for each of the six assets (don't forget to add the constant for each). There are three steps for each of them:
1. Estimate the model (you can use OLS from statsmodels for this) and print out the summary.
2. Test the restriction $\alpha = 0$ and $\beta = 1$ **individually**.
3. Test the restriction $\alpha = 0$ and $\beta = 1$ **jointly**.
The three code cells below are for Exxon. Follow the same pattern for: GE, IBM, Microsoft, Walmart, Gold. At the end, insert a new Markdown cell and discuss how the $\beta$ for Gold differs from the rest and what it means.

**Important note**: All the returns data have a missing value at the bottom. In addition, the gold data from Yahoo! Finance also has additional missing observations. Don't delete any of these manually. Instead, you can take care of this by using an argument inside the OLS function. Look up the documentation here: https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html#statsmodels.regression.linear_model.OLS

In [6]:
# Estimate for Exxon
X = ret_2['^GSPC'] - tbill['TB4WK']
X = sm.add_constant(X)
X = X.rename(columns={0 : 'x'})
y = ret_2['XOM'] - tbill['TB4WK']

model = sm.OLS(y, X, missing='drop')
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     71.93
Date:                Tue, 25 Jul 2023   Prob (F-statistic):           8.36e-15
Time:                        12:49:59   Log-Likelihood:                 303.59
No. Observations:                 180   AIC:                            -603.2
Df Residuals:                     178   BIC:                            -596.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0009      0.003     -0.253      0.8

In [7]:
# Test the individual restrictions for Exxon

# Alpha
print("Test for 𝛼 = 0:")
hypothesis_alpha = 'const = 0' 
test_alpha = results.t_test(hypothesis_alpha)
print(test_alpha)

# Beta
print("\nTest for 𝛽 = 1:")
hypothesis_beta = 'x = 1'  
test_beta = results.t_test(hypothesis_beta)
print(test_beta)

Test for 𝛼 = 0:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0009      0.003     -0.253      0.801      -0.008       0.006

Test for 𝛽 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.7071      0.083     -3.512      0.001       0.543       0.872


In [8]:
# Test the joint restriction for Exxon
hypothesis_joint = ('const = 0, x = 1')
test_alpha = results.f_test(hypothesis_joint)
print("Joint Test:")
print(test_alpha)

Joint Test:
<F test: F=6.385048409822016, p=0.0020987634634270828, df_denom=178, df_num=2>


In [9]:
# Estimate for GE
X = ret_2['^GSPC'] - tbill['TB4WK']
X = sm.add_constant(X)
X = X.rename(columns={0 : 'x'})
y = ret_2['GE'] - tbill['TB4WK']

model = sm.OLS(y, X, missing='drop')
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.445
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     142.5
Date:                Tue, 25 Jul 2023   Prob (F-statistic):           1.64e-24
Time:                        12:50:03   Log-Likelihood:                 239.29
No. Observations:                 180   AIC:                            -474.6
Df Residuals:                     178   BIC:                            -468.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0106      0.005     -2.185      0.0

In [10]:
# Test the individual restrictions for GE

# Alpha
print("Test for 𝛼 = 0:")
hypothesis_alpha = 'const = 0' 
test_alpha = results.t_test(hypothesis_alpha)
print(test_alpha)

# Beta
print("\nTest for 𝛽 = 1:")
hypothesis_beta = 'x = 1'  
test_beta = results.t_test(hypothesis_beta)
print(test_beta)

# Test the joint restriction for GE
hypothesis_joint = ('const = 0, x = 1')
test_alpha = results.f_test(hypothesis_joint)
print("\nJoint Test:")
print(test_alpha)

Test for 𝛼 = 0:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0106      0.005     -2.185      0.030      -0.020      -0.001

Test for 𝛽 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             1.4228      0.119      3.547      0.000       1.188       1.658

Joint Test:
<F test: F=7.896054039446618, p=0.0005181654604467371, df_denom=178, df_num=2>


In [11]:
# Estimate for IBM
X = ret_2['^GSPC'] - tbill['TB4WK']
X = sm.add_constant(X)
X = X.rename(columns={0 : 'x'})
y = ret_2['IBM'] - tbill['TB4WK']

model = sm.OLS(y, X, missing='drop')
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.338
Method:                 Least Squares   F-statistic:                     92.32
Date:                Tue, 25 Jul 2023   Prob (F-statistic):           7.17e-18
Time:                        12:50:07   Log-Likelihood:                 289.05
No. Observations:                 180   AIC:                            -574.1
Df Residuals:                     178   BIC:                            -567.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0005      0.004     -0.146      0.8

In [12]:
# Test the individual restrictions for IBM

# Alpha
print("Test for 𝛼 = 0:")
hypothesis_alpha = 'const = 0' 
test_alpha = results.t_test(hypothesis_alpha)
print(test_alpha)

# Beta
print("\nTest for 𝛽 = 1:")
hypothesis_beta = 'x = 1'  
test_beta = results.t_test(hypothesis_beta)
print(test_beta)

# Test the joint restriction for IBM
hypothesis_joint = ('const = 0, x = 1')
test_alpha = results.f_test(hypothesis_joint)
print("\nJoint Test:")
print(test_alpha)

Test for 𝛼 = 0:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0005      0.004     -0.146      0.884      -0.008       0.007

Test for 𝛽 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.8685      0.090     -1.455      0.148       0.690       1.047

Joint Test:
<F test: F=1.107139341945525, p=0.3327680276896564, df_denom=178, df_num=2>


In [13]:
# Estimate for MSFT
X = ret_2['^GSPC'] - tbill['TB4WK']
X = sm.add_constant(X)
X = X.rename(columns={0 : 'x'})
y = ret_2['MSFT'] - tbill['TB4WK']

model = sm.OLS(y, X, missing='drop')
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.401
Method:                 Least Squares   F-statistic:                     121.0
Date:                Tue, 25 Jul 2023   Prob (F-statistic):           8.35e-22
Time:                        12:50:08   Log-Likelihood:                 282.83
No. Observations:                 180   AIC:                            -561.7
Df Residuals:                     178   BIC:                            -555.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0064      0.004      1.699      0.0

In [14]:
# Test the individual restrictions for MSFT

# Alpha
print("Test for 𝛼 = 0:")
hypothesis_alpha = 'const = 0' 
test_alpha = results.t_test(hypothesis_alpha)
print(test_alpha)

# Beta
print("\nTest for 𝛽 = 1:")
hypothesis_beta = 'x = 1'  
test_beta = results.t_test(hypothesis_beta)
print(test_beta)

# Test the joint restriction for MSFT
hypothesis_joint = ('const = 0, x = 1')
test_alpha = results.f_test(hypothesis_joint)
print("\nJoint Test:")
print(test_alpha)

Test for 𝛼 = 0:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0064      0.004      1.699      0.091      -0.001       0.014

Test for 𝛽 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             1.0293      0.094      0.313      0.754       0.845       1.214

Joint Test:
<F test: F=1.5741837825661604, p=0.21004696310360266, df_denom=178, df_num=2>


In [15]:
# Estimate for WMT
X = ret_2['^GSPC'] - tbill['TB4WK']
X = sm.add_constant(X)
X = X.rename(columns={0 : 'x'})
y = ret_2['WMT'] - tbill['TB4WK']

model = sm.OLS(y, X, missing='drop')
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     16.20
Date:                Tue, 25 Jul 2023   Prob (F-statistic):           8.42e-05
Time:                        12:50:08   Log-Likelihood:                 298.77
No. Observations:                 180   AIC:                            -593.5
Df Residuals:                     178   BIC:                            -587.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0036      0.003      1.050      0.2

In [16]:
# Test the individual restrictions for WMT

# Alpha
print("Test for 𝛼 = 0:")
hypothesis_alpha = 'const = 0' 
test_alpha = results.t_test(hypothesis_alpha)
print(test_alpha)

# Beta
print("\nTest for 𝛽 = 1:")
hypothesis_beta = 'x = 1'  
test_beta = results.t_test(hypothesis_beta)
print(test_beta)

# Test the joint restriction for WMT
hypothesis_joint = ('const = 0, x = 1')
test_alpha = results.f_test(hypothesis_joint)
print("\nJoint Test:")
print(test_alpha)

Test for 𝛼 = 0:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0036      0.003      1.050      0.295      -0.003       0.010

Test for 𝛽 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.3447      0.086     -7.651      0.000       0.176       0.514

Joint Test:
<F test: F=29.286846225780756, p=1.0101044785380091e-11, df_denom=178, df_num=2>


In [17]:
# Estimate for SP500
X = ret_2['^GSPC'] - tbill['TB4WK']
X = sm.add_constant(X)
X = X.rename(columns={0 : 'x'})
y = ret_2['GC=F'] - tbill['TB4WK']

model = sm.OLS(y, X, missing='drop')
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.6646
Date:                Tue, 25 Jul 2023   Prob (F-statistic):              0.416
Time:                        12:50:10   Log-Likelihood:                 196.35
No. Observations:                 128   AIC:                            -388.7
Df Residuals:                     126   BIC:                            -383.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0068      0.005      1.458      0.1

In [18]:
# Test the individual restrictions for SP500

# Alpha
print("Test for 𝛼 = 0:")
hypothesis_alpha = 'const = 0' 
test_alpha = results.t_test(hypothesis_alpha)
print(test_alpha)

# Beta
print("\nTest for 𝛽 = 1:")
hypothesis_beta = 'x = 1'  
test_beta = results.t_test(hypothesis_beta)
print(test_beta)

# Test the joint restriction for WMT
hypothesis_joint = ('const = 0, x = 1')
test_alpha = results.f_test(hypothesis_joint)
print("\nJoint Test:")
print(test_alpha)

Test for 𝛼 = 0:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0068      0.005      1.458      0.147      -0.002       0.016

Test for 𝛽 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0923      0.113     -8.012      0.000      -0.132       0.317

Joint Test:
<F test: F=32.62106618659261, p=3.833354297104192e-12, df_denom=126, df_num=2>


The Beta of Gold is really low, as its value tends to move independently of the our selected stocks and index. Company GE, IBM, MSFT has the number greater than Gold reflecting their sensitivity to Gold market movements. Company GE, WMT and SP index has the lower p-value show a strong statistical relationship with Gold inedex. Company WMT has a higher p-value tends to be less volatile Gold, however it could be affected by the selected time period. The F-statistic of all selected stocks' returns have significant relationship with the Gold return.